# MediaPipe Object Detection for AWS Lambda

This notebook demonstrates how to use **MediaPipe Tasks** for Object Detection and how to structure the code for deployment on **AWS Lambda** using container images.

## 1. Setup Environment
First, install the required libraries. For AWS Lambda, these should be included in your `requirements.txt`.

In [ ]:
!pip install mediapipe opencv-python

## 2. Download Model
MediaPipe Tasks require a `.tflite` model file. We will use the EfficientDet-Lite0 model.

In [ ]:
!wget -q -O efficientdet.tflite https://storage.googleapis.com/mediapipe-models/object_detector/efficientdet_lite0/metadata/1/efficientdet_lite0.tflite

## 3. Object Detection Function
This is the core logic that will be reused in the Lambda handler.

In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import numpy as np

def detect_objects(image_path, model_path='efficientdet.tflite'):
    # Create options
    base_options = python.BaseOptions(model_asset_path=model_path)
    options = vision.ObjectDetectorOptions(base_options=base_options,
                                          score_threshold=0.5)
    detector = vision.ObjectDetector.create_from_options(options)

    # Load image
    image = mp.Image.create_from_file(image_path)

    # Detect
    detection_result = detector.detect(image)
    
    return detection_result

# Test locally (Make sure to provide a sample image)
# results = detect_objects('sample.jpg')
# print(results)

## 4. Structuring for AWS Lambda
When moving to Lambda, you need to handle base64 images and return JSON results.

In [ ]:
import json
import base64
import io
from PIL import Image

def lambda_handler(event, context):
    try:
        # 1. Parse image from event (base64)
        body = json.loads(event['body'])
        image_data = base64.b64decode(body['image'])
        
        # Save temporarily to /tmp as MediaPipe create_from_file expects a path
        # Or use create_from_format if processing memory buffers
        with open('/tmp/input.jpg', 'wb') as f:
            f.write(image_data)

        # 2. Run Detection
        results = detect_objects('/tmp/input.jpg')

        # 3. Format Response
        detections = []
        for detection in results.detections:
            bbox = detection.bounding_box
            category = detection.categories[0]
            detections.append({
                "label": category.category_name,
                "score": float(category.score),
                "bbox": [bbox.origin_x, bbox.origin_y, bbox.width, bbox.height]
            })

        return {
            'statusCode': 200,
            'body': json.dumps({'detections': detections})
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }